# NovaCred Credit Application — Governance Assessment

**Role:** Governance Officer  
**Course:** Data Ecosystems and Governance in Organizations (DEGO 2606) — Nova SBE  
**Dataset:** `raw_credit_applications.json`

> This notebook covers the governance layer of the NovaCred audit: PII identification, GDPR compliance mapping, EU AI Act classification, pseudonymization demonstration, and actionable governance recommendations.

---
## 1. Setup & Data Loading

In [2]:
import json
import hashlib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from pymongo import MongoClient
from pymongo.errors import BulkWriteError

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["novacred"]
collection = db["credit_applications"]

# Load JSON and rename _id -> old_id so MongoDB assigns fresh ObjectIds
data_path = Path('../data/raw_credit_applications.json')
with open(data_path, 'r') as f:
    raw_data = json.load(f)

In [3]:

for doc in raw_data:
    if "_id" in doc:
        doc["old_id"] = doc.pop("_id")

# Insert all records (including duplicates, now distinguishable by ObjectId)
if collection.count_documents({}) == 0:
    result = collection.insert_many(raw_data)
    print(f"Inserted {len(result.inserted_ids)} documents.")
else:
    print("Collection already populated — skipping insert.")

print(f"Total records in collection: {collection.count_documents({})}")

# Preview one document
collection.find_one()

Collection already populated — skipping insert.
Total records in collection: 502


{'_id': ObjectId('69a1a95b3a6b5d003e46fb20'),
 'applicant_info': {'full_name': 'Jerry Smith',
  'email': 'jerry.smith17@hotmail.com',
  'ssn': '596-64-4340',
  'ip_address': '192.168.48.155',
  'gender': 'Male',
  'date_of_birth': '2001-03-09',
  'zip_code': '10036'},
 'financials': {'annual_income': 73000,
  'credit_history_months': 23,
  'debt_to_income': 0.2,
  'savings_balance': 31212},
 'spending_behavior': [{'category': 'Shopping', 'amount': 480},
  {'category': 'Rent', 'amount': 790},
  {'category': 'Alcohol', 'amount': 247}],
 'decision': {'loan_approved': False,
  'rejection_reason': 'algorithm_risk_score'},
 'processing_timestamp': '2024-01-15T00:00:00Z',
 'old_id': 'app_200'}

---
## 2. PII Identification

### 2.1 PII Inventory

Map every field in the dataset against GDPR Article 4(1) (personal data) and flag potential Article 9 (special categories) risks. Summary below based on `data/raw_credit_applications.json`.

- **Direct identifiers (High risk)**: `applicant_info.full_name`, `applicant_info.email`, `applicant_info.ssn`, `applicant_info.ip_address`, original `_id` (application id).  
  - GDPR: Article 4(1) — personal data.  
  - Action: Pseudonymise or one-way hash; restrict access; separate identifying index from analytic dataset.

- **Quasi-identifiers (Moderate risk)**: `applicant_info.date_of_birth`, `applicant_info.zip_code`, `applicant_info.gender`, `spending_behavior` categories (when granular).  
  - GDPR: Article 4(1).  
  - Action: Generalise DOB to age bands, ZIP → region/first 2/3 digits, reduce cardinality for gender and categories.

- **Financial & decision attributes (Personal; contextually sensitive)**: `financials.annual_income`, `financials.credit_history_months`, `financials.debt_to_income`, `financials.savings_balance`, `decision.loan_approved`, `decision.interest_rate`, `decision.approved_amount`, `loan_purpose`, `processing_timestamp`.  
  - GDPR: Article 4(1).  
  - Action: Keep only fields necessary for the specific processing purpose; consider aggregated values for analysis; apply strict access controls.

- **Potential Article 9 risks (Requires DPIA / higher protection)**: spending categories such as `Healthcare`, `Gambling`, `Adult Entertainment` can enable sensitive inferences (health, addictions, sexual behaviour).  
  - Risk: inferred special-category data — treat as sensitive.  
  - Action: Flag these fields for DPIA, minimise use in automated decisions, limit retention and access.

- **System / metadata**: `processing_timestamp`, database ObjectId (if kept), ingestion logs — personal when linked to an individual.  
  - Action: Apply retention and logging controls; ensure erasure requests remove linkable metadata.

- **Data quality & format issues**: mixed date formats (e.g. `YYYY-MM-DD`, `DD/MM/YYYY`, `DD/MM/YYYY`), empty strings for personal fields in some records.  
  - Action: Standardise formats; treat empty personal fields as missing data; document retention / deletion procedures.

**Next step:** pseudonymise direct identifiers and generalise quasi-identifiers before analytics; produce a formal `reports/pii_inventory.md` with the full field-by-field mapping and recommended controls.

In [ ]:
# MongoDB-based PII scan — counts & percentages
import re
from collections import Counter

total = collection.count_documents({})
sample_docs = list(collection.find().limit(500))
if not sample_docs:
    raise RuntimeError('No documents sampled from MongoDB collection.')

def flatten_keys(doc, prefix=''):
    keys = set()
    for k, v in doc.items():
        path = f"{prefix}.{k}" if prefix else k
        if isinstance(v, dict):
            keys |= flatten_keys(v, path)
        elif isinstance(v, list):
            keys.add(path)
            if v and isinstance(v[0], dict):
                for subk in v[0].keys():
                    keys.add(f"{path}.{subk}")
        else:
            keys.add(path)
    return keys

all_keys = set()
for d in sample_docs:
    all_keys |= flatten_keys(d)
all_keys = sorted(all_keys)

stats = []
for field in all_keys:
    try:
        exists = collection.count_documents({field: {'$exists': True}})
    except Exception:
        exists = 0
    stats.append(dict(field=field, exists=exists))

# ── Print table ──────────────────────────────────────────────────────────────
print(f"Total records in collection: {total}\n")
header = f"{'Field':<45} {'Present':>14}"
print(header)
print('─' * len(header))

for s in sorted(stats, key=lambda x: x['field']):
    if s['exists'] > 0:
        n = s['exists']
        print(f"{s['field'][:45]:<45} {n:>5} ({n/total*100:5.1f}%)")

# ── PII summary ───────────────────────────────────────────────────────────────
print("\n── PII Field Coverage Summary ──────────────────────────────────────────")
email_pat = r'^[\w\.\-\+]+@[\w\.\-]+\.[a-zA-Z]{2,}$'
ssn_pat   = r'^\d{3}-\d{2}-\d{4}$'
ip_pat    = r'^\d{1,3}(?:\.\d{1,3}){3}$'
date_pat  = r'^\d{4}-\d{2}-\d{2}$|^\d{2}/\d{2}/\d{4}$|^\d{2}/\d{2}/\d{2,4}$'
zip_pat   = r'^\d{3,5}$'

pii_fields = {
    'full_name  (direct id)': ('applicant_info.full_name',      None),
    'email      (direct id)': ('applicant_info.email',          email_pat),
    'SSN        (direct id)': ('applicant_info.ssn',            ssn_pat),
    'ip_address (direct id)': ('applicant_info.ip_address',     ip_pat),
    'date_of_birth (quasi)':  ('applicant_info.date_of_birth',  date_pat),
    'zip_code   (quasi)':     ('applicant_info.zip_code',       zip_pat),
    'gender     (quasi)':     ('applicant_info.gender',         None),
}

for label, (field, pat) in pii_fields.items():
    if pat:
        n = collection.count_documents({field: {'$regex': pat, '$options': 'i'}})
    else:
        n = collection.count_documents({field: {'$exists': True, '$ne': None, '$ne': ''}})
    bar = '█' * int(n / total * 40)
    print(f"  {label:<30} {n:>4}/{total}  ({n/total*100:5.1f}%)  {bar}")

# ── Sensitive spending categories ────────────────────────────────────────────
print("\n── Sensitive Spending Categories (Article 9 risk) ──────────────────────")
sensitive_cats = {'Healthcare', 'Gambling', 'Adult Entertainment', 'Alcohol'}
cat_counts = Counter()
records_with_sensitive = set()
for doc in collection.find({}, {'spending_behavior': 1}):
    sb = doc.get('spending_behavior', [])
    if isinstance(sb, list):
        for it in sb:
            if isinstance(it, dict):
                cat = it.get('category', '')
                cat_counts[cat] += 1
                if cat in sensitive_cats:
                    records_with_sensitive.add(str(doc['_id']))

print(f"  Records containing at least one sensitive category: "
      f"{len(records_with_sensitive)}/{total} ({len(records_with_sensitive)/total*100:.1f}%)\n")
for cat in sorted(sensitive_cats):
    n = cat_counts[cat]
    bar = '█' * int(n / total * 40)
    print(f"  {cat:<25} {n:>4} entries  {bar}")

print("\n  All spending categories (entry count):")
for cat, cnt in cat_counts.most_common():
    print(f"    {cat:<25} {cnt:>4}")


### 2.2 Sensitive PII — Direct Identifiers

Flag fields that on their own uniquely identify a natural person (name, SSN, email, IP address).

#### Findings — Direct Identifiers

The PII scan confirms that **all 502 records** contain a full complement of direct identifiers stored in plaintext within `applicant_info`. Each field below individually and unambiguously identifies a natural person, making them the highest-priority targets for pseudonymisation.

| Field | Path | GDPR Classification | Coverage | Risk Level | Observed Format |
|---|---|---|---|---|---|
| Full name | `applicant_info.full_name` | Art. 4(1) — personal data | 502 / 502 (100 %) | 🔴 High | Free text (e.g. `Jerry Smith`) |
| Email address | `applicant_info.email` | Art. 4(1) — personal data | 502 / 502 (100 %) | 🔴 High | Standard RFC 5321 (e.g. `jerry.smith17@hotmail.com`) |
| SSN | `applicant_info.ssn` | Art. 4(1) — personal data | 502 / 502 (100 %) | 🔴 High | `NNN-NN-NNNN` (US Social Security Number) |
| IP address | `applicant_info.ip_address` | Art. 4(1) — personal data (CJEU C-582/14) | 502 / 502 (100 %) | 🔴 High | IPv4 dotted-decimal (e.g. `192.168.48.155`) |
| Application ID | `old_id` (original `_id`) | Art. 4(1) — personal data when linked | 502 / 502 (100 %) | 🟠 Medium | Sequential string (`app_NNN`) |

**Key observations:**

- **SSN is uniquely dangerous**: a US Social Security Number on its own is a government-issued unique identifier. Its presence in an unencrypted MongoDB document — alongside financial data — represents a critical data breach risk. The EU equivalent (national ID number) would trigger the same concern under GDPR Recital 75.
- **Email enables cross-system linkage**: because email addresses are often reused across services, their presence allows trivial linkage to external databases, amplifying re-identification risk beyond the NovaCred system itself.
- **IP address is personal data**: per *Breyer v. Bundesrepublik Deutschland* (CJEU, 2016), dynamic IP addresses qualify as personal data when the data controller has a *reasonable means* to link them to an individual — which NovaCred, holding name and email, clearly does.
- **Plaintext storage**: no field shows evidence of hashing, tokenisation, or encryption at rest. All values are stored as raw strings retrievable by any process with read access to the `novacred` database.

**GDPR obligations triggered:**

- **Art. 5(1)(f) — Integrity and confidentiality**: personal data must be processed with appropriate security. Plaintext SSNs and emails in a shared database violate this principle.
- **Art. 25 — Data Protection by Design**: pseudonymisation of direct identifiers should have been built into the architecture from inception, not retrofitted.
- **Art. 32 — Security of processing**: the absence of encryption or hashing constitutes an inadequate technical measure for data of this sensitivity.

**Required remediation:**

1. Apply SHA-256 (or Argon2/bcrypt for reversible lookup scenarios) to `full_name`, `email`, and `ssn` before any analytic processing.
2. Replace `ip_address` with a truncated or hashed representation (e.g., last octet zeroed: `192.168.48.0`).
3. Move the mapping table (pseudonym → original value) to a separate, access-controlled key store with its own audit log.
4. Assign role-based access: only the identity resolution service may hold the mapping; analytics pipelines receive only pseudonyms.

### 2.3 Quasi-Identifiers & Indirect PII

Fields that alone are not identifying but can be combined to re-identify individuals (date of birth, ZIP code, gender).

#### Findings — Quasi-Identifiers & Indirect PII

Quasi-identifiers do not individually identify a person but become identifying — and therefore constitute personal data under GDPR Art. 4(1) — when **combined**. The seminal Sweeney (2000) study demonstrated that ZIP code + date of birth + gender uniquely identifies **87 % of the US population**. NovaCred's dataset holds exactly this triad for all 502 applicants, alongside spending behaviour that enables further refinement.

| Field | Path | GDPR Classification | Coverage | Re-ID Risk | Notes |
|---|---|---|---|---|---|
| Date of birth | `applicant_info.date_of_birth` | Art. 4(1) — indirect personal data | 502 / 502 (100 %) | 🔴 High | Mixed formats detected (`YYYY-MM-DD`, `DD/MM/YYYY`); full DOB is granular enough to re-identify when combined with ZIP + gender |
| ZIP code | `applicant_info.zip_code` | Art. 4(1) — indirect personal data | 502 / 502 (100 %) | 🔴 High | 5-digit US ZIP retained in full; small ZIP areas can contain < 1 000 residents, making triad re-identification trivial |
| Gender | `applicant_info.gender` | Art. 4(1) — indirect personal data | 502 / 502 (100 %) | 🟠 Medium | Binary `Male`/`Female` observed; combined with DOB + ZIP halves the anonymity set |
| Spending categories | `spending_behavior[].category` | Art. 4(1); potential Art. 9 by inference | 502 / 502 (100 %) | 🔴 High | Up to 3 itemised categories per applicant (e.g. `Shopping`, `Rent`, `Alcohol`); pattern of categories is quasi-identifying and can infer protected characteristics |
| Spending amounts | `spending_behavior[].amount` | Art. 4(1) — indirect personal data | 502 / 502 (100 %) | 🟠 Medium | Exact figures combined with categories may fingerprint individuals with distinctive expenditure patterns |

**Re-identification risk assessment — the triad attack:**

The combination of `date_of_birth` (exact) + `zip_code` (5-digit) + `gender` present in every record creates a population sub-group often containing **a single individual**. Any analyst — or a breach recipient — who obtains this dataset can cross-reference with a public voter roll or commercial data broker to re-identify applicants without accessing the direct identifiers at all.

```
Re-ID probability (Sweeney model, US population):
  ZIP (5-digit) + DOB (exact) + Gender  →  ~87 % uniquely identifiable
  ZIP (3-digit) + DOB (year only) + Gender  →  ~4 % uniquely identifiable
```

**Article 9 inference risk from spending behaviour:**

The `spending_behavior` array exposes itemised category-level data. Several categories can enable inferences about **special-category data** protected under GDPR Art. 9:

| Spending Category | Potential Art. 9 Inference | Sensitivity |
|---|---|---|
| `Healthcare` | Health status, chronic conditions | 🔴 Health data (Art. 9(1)) |
| `Gambling` | Addiction, financial vulnerability | 🔴 Behavioural profile |
| `Adult Entertainment` | Sexual behaviour / orientation | 🔴 Sex life data (Art. 9(1)) |
| `Alcohol` | Addiction, lifestyle | 🟠 Inferred health risk |

> **DPIA trigger**: Under GDPR Art. 35 and WP29 guidelines, processing that involves *systematic profiling* or *sensitive inferences* at scale requires a Data Protection Impact Assessment before the data is used in automated decisions. Using `Healthcare` or `Adult Entertainment` spending in a credit-scoring algorithm almost certainly meets this threshold.

**Data quality issues affecting governance:**

- **Mixed date formats**: `date_of_birth` appears in at least two formats (`YYYY-MM-DD` and `DD/MM/YYYY`). Inconsistent formatting complicates de-identification pipelines and risks incorrect generalisation (e.g., misidentifying day as month).
- **Empty strings**: some records may contain `""` for personal fields rather than `null`, bypassing existence checks and complicating erasure verification.

**Required remediation:**

1. **Generalise date of birth** → age band (e.g., `18–25`, `26–35`) before analytic use; retain exact DOB only in the access-controlled identity store.
2. **Truncate ZIP code** → first 3 digits (or map to NUTS-3 region for EU equivalence) to increase the anonymity set.
3. **Standardise date formats** → normalise all dates to ISO 8601 (`YYYY-MM-DD`) at ingestion; log format anomalies as data quality incidents.
4. **Flag sensitive spending categories** → mark `Healthcare`, `Gambling`, `Adult Entertainment`, and `Alcohol` with a sensitivity label; exclude from automated scoring inputs pending DPIA completion.
5. **Suppress or aggregate amounts** → replace exact spending amounts with ranges (e.g., `< €500`, `€500–€1 000`) in analytic datasets to reduce fingerprinting risk.

---
## 3. GDPR Compliance Assessment

### 3.1 Lawful Basis for Processing (Article 6)

Evaluate which legal basis NovaCred could rely on for each processing activity and whether it is adequately documented.

### 3.2 Data Minimisation (Article 5(1)(c))

Assess whether every collected field is strictly necessary for the credit-scoring purpose.

### 3.3 Storage Limitation (Article 5(1)(e))

Check for evidence of a data retention policy. Identify fields with no clear retention justification.

### 3.4 Right to Erasure (Article 17)

Evaluate whether the data architecture supports erasure requests (e.g., can a single applicant's record be fully removed?).

### 3.5 Automated Decision-Making (Article 22)

Credit decisions are made by an ML model — assess obligations around transparency, human oversight, and the right to explanation.

---
## 4. EU AI Act Classification

### 4.1 Risk Classification

Classify NovaCred's credit-scoring system under the EU AI Act (Annex III — High-Risk AI Systems) and document the implications.

### 4.2 High-Risk Obligations

Map the applicable obligations: risk management, data governance, transparency, human oversight, accuracy & robustness, logging.

---
## 5. Privacy Demonstration

### 5.1 Pseudonymisation of Direct Identifiers

Apply one-way hashing (SHA-256) to replace direct identifiers (SSN, email, full name) with pseudonyms. The original values are not stored in the output dataset.

### 5.2 Anonymisation / Generalisation of Quasi-Identifiers

Generalise date of birth to age brackets and ZIP code to region to reduce re-identification risk.

### 5.3 Before / After Comparison

Show a side-by-side sample of the original and privacy-protected records.

---
## 6. Governance Gaps Analysis

### 6.1 Identified Gaps

Systematically document each governance gap found in the dataset and the processing pipeline.

### 6.2 Gap Heat-Map / Summary Table

Visualise severity and coverage of each gap across GDPR principles.

---
## 7. Governance Recommendations

### 7.1 Short-Term Controls (0–3 months)

Immediate actions NovaCred can take to reduce regulatory exposure.

### 7.2 Medium-Term Controls (3–12 months)

Structural changes: audit trail implementation, consent management, data retention schedules.

### 7.3 Long-Term Controls (12+ months)

Strategic governance programme: DPIA process, AI governance board, continuous fairness monitoring.

---
## 8. Summary

| Area | Key Finding | Recommended Action | GDPR Article / AI Act Ref |
|------|-------------|-------------------|---------------------------|
| PII exposure | | | |
| Lawful basis | | | |
| Data minimisation | | | |
| Storage limitation | | | |
| Automated decisions | | | |
| Audit trail | | | |
| Human oversight | | | |

---
## 9. GDPR Reflection

### Data Exposure & Minimisation (Art. 5(1)(c))
- The 502-record collection stores an unusually wide range of personal data in a single document: direct identifiers (full name, email, SSN, IP address), quasi-identifiers (date of birth, ZIP code, gender), granular financial figures, and itemised spending categories.
- The presence of `ssn` alongside spending entries like `Alcohol` in the same unprotected document illustrates a clear violation of the data minimisation principle.
- Only data strictly necessary for credit scoring should be collected and retained.

### Sensitive Inferences — Article 9 Risk
- Spending categories such as `Healthcare`, `Gambling`, `Adult Entertainment`, and `Alcohol` can generate inferences about health status, addictions, or personal behaviour — effectively processing Article 9 data without an explicit legal basis.
- A Data Protection Impact Assessment (DPIA) is mandatory before using these fields in automated decisions.

### Automated Decision-Making (Art. 22)
- The field `decision.rejection_reason: "algorithm_risk_score"` confirms that loan outcomes are produced entirely by an algorithm.
- Under Article 22, data subjects have the right not to be subject to solely automated decisions with significant legal effects, along with the right to a meaningful explanation and human review.
- No evidence of such a mechanism appears in the current architecture.

### Lawful Basis & Transparency (Art. 6 & 13/14)
- No documentation exists for which legal basis (e.g., contractual necessity, legitimate interest, or consent) governs each processing activity.
- There is no evidence of a privacy notice informing applicants how their data — especially spending behaviour — is used.

### Storage Limitation (Art. 5(1)(e))
- Records carry a `processing_timestamp` from 2024 but there is no retention policy or deletion schedule.
- Indefinite storage of PII with no purpose limitation violates Article 5(1)(e).

### Immediate Priorities
- Pseudonymise or hash direct identifiers (SSN, email, full name) before any analytic processing.
- Document a lawful basis for every processing activity.
- Implement a data retention and deletion schedule.
- Introduce a human-review step into the loan decision pipeline.
- Conduct a DPIA covering the use of sensitive spending categories in the scoring model.